
#<계획표>
>
5월  9일(월) ~ 13일(금) 개인 분석 

17일(화) 조합 후 레포트 작성

5월 18일(수) ~ 19일(목) ppt 및 대본 작업

5월 20일(금) 오전까지 프로젝트 최종 점검 제출

5월 23일(월) 발표


#프로젝트 가이드라인 (변치웅 강사님의 꿀팁!)
1. 프로젝트 수행목적, 동기
2. 사용할 데이터의 변수 설명 (수집경로, 단위 포함)
3. 최소 10,000개 샘플 사용
4. 모델 선택 및 최적화과정 포함
5. 솔루션 제공 및 문제점 도출
- 상기 내용은 강제하는 내용은 아니지만 포함될 경우 좋은 프로젝트로 평가받는 요소들 입니다.
- 데이터를 다각화 하여 보는게 중요


#<프로젝트>

1. 문제정의 (선정이유)
2. 데이터 수집
3. 데이터 탐색 (이슈사항 검토 ex.생존율에는 어떤 요소가 영향을 주는지 / 분포는 어떻게 되는지)
4. 데이터 전처리
5. 모델링 (데이터 분포 분석 -> 상관관계 파악 -> 시각화)
6. 결과

>대주제 : 매출에 영향을 끼치는 요인

매출의 기준: 최종 결제까지 간 사람
(type == payment and state == completed) - (type == refund and state == completed )

각 문제별로 새 데이터프레임 만들어서 적용 


- 메인(+서브, 강의명) 카테고리
1. 10개의 카테고리 중 환불된 강의 카테고리는?
(어떤 강의들이 환불되는지?? 어떤 특징??)

- 할인금액
1. 얼마 할인해줬을 때, 가장 많은 결제를 이끌어냈는지
2. 같은 카테고리기준으로 할인 혜택을 받은 사람과 안받은 사람의 비율

- 재결재율:
1. 웰컴 할인쿠폰을 받은 사람 중 재결재한사람(신규 유입에 쿠폰의 영향?)
2. 동일한 결제 수단으로 결제 여부

- 강의 가격: 
1. 같은 카테고리 안에서 서브 카테고리별로 높은 가격 vs 낮은가격 어떤 강의 인기? 
-> .describe()사용 (실구매가, 정가 비교)



( 시간되면 )모델링에 추가
1. 어느 결제대행사로 결제하는지 + 이벤트/할인 많이 해서? 
2. 수강율이 낮은 강의 프로모션 제안 





#1. 문제정의

#2. 데이터 수집


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
df=pd.read_excel('/content/drive/MyDrive/미니프로젝트/파이널프로젝트_RAW_210329_210926.xlsx')
df = df[['customer_id', 'course_id', 'type', 'state' , 'course_title', 'category_title', 'format', 'completed_at', 'transaction_amount', 'coupon_title', 'coupon_discount_amount', 'sale_price', 'pg', 'method', 'subcategory_title', 'marketing_start_at']]
df.head()


,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,NaN,VBANK,개발자 커리어,NaT
1,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
3,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14


#3. 데이터 탐색

>컬럼 설명

- (삭제) id                      << 버려도 됨..??? 

- customer_id             << 중복있음!! +순수 고객수 파악하려면 전처리 필요

- course_id               << 어떤 강의 듣는지(빈 값은 폐강으로 추측)

- type                    << 해당 강좌의 현재 결제 진행상태(order는 장바구니, payment가 최종 결제)

- state                   << type의 진행상황

- course_title            << 강의 제목

- category_title          << 강의 카테고리 제목

- (삭제 보류)format                  << 강의 과정 분류 종류

- completed_at            << 강의 결제 완료 시각

- transaction_amount      << 실 결제 금액

- coupon_title            << 쿠폰명

- coupon_discount_amount  << 할인된 금액

- sale_price              << 판매 금액

- (삭제) tax_free_amount         << 면세액 

- pg                      << 결제대행사 (빈칸은 그 외의 결제대행사)

- method                  << 결제 방법

- subcategory_title       << 강의 부 카테고리 제목

- marketing_start_at      << 해당 강의가 열린날 (일단위로 기록됨)


____
<유의할 사항>
- 쿠폰 말고도 이벤트같은 다른 조건에 의한 할인도 있다


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105419 entries, 0 to 105418
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   id                      105419 non-null  int64         
 1   customer_id             105416 non-null  float64       
 2   course_id               100212 non-null  float64       
 3   type                    105419 non-null  object        
 4   state                   105419 non-null  object        
 5   course_title            105403 non-null  object        
 6   category_title          105395 non-null  object        
 7   format                  105403 non-null  object        
 8   completed_at            105419 non-null  datetime64[ns]
 9   transaction_amount      105419 non-null  int64         
 10  coupon_title            31817 non-null   object        
 11  coupon_discount_amount  31817 non-null   float64       
 12  sale_price              104291

# 분석 주제 토의

최적의 가성비 할인율은 얼마?

판매 금액의 얼마정도를 할인했을 때 결제율이 상승하는지

sale_price , transaction_amount를 통해 할인율을 10% 단위로 분류

할인율에 따라 결제 수를 비교

+카테고리별, 




해비유저 vs 라이트 유저 둘 중 누가 더 매출 많이 올려주는지

두 그룹의 결제 빈도

---


카테고리 별 관심도 >> 할인 여부 

마케팅 시작일과 수강시작일의 관계

웰컴쿠폰 사용자가 재결제하는지

---

마케팅을 시작하고 수강 완료 시간 사이에 코스

코스를 카테고리별로 분류해서 요금의 평균을 어느 강의가 가격대 분석 >> 가격대에 비교했을때 결제율이 차이나는지

--


서브 카테고리를 통해 강의 주제를 분류하고 같은 종류의 강의에서 쿠폰의 여부 결제 판단에 영향을 미치는지


--


결제 금액과 카테고리의 상관관계 + 있다면 쿠폰이 영향을 주는지 + 결제대행사가 결제(할인을 준다던가) 영향을 주는지 

마케팅 시작일 할때 강의수강률이 올라가는지?


#4. 데이터 전처리

>결측치 제거

In [53]:
#결측치 제거
df = df.dropna(subset=['category_title'])
df.isna().sum()

customer_id                   0
course_id                  5183
type                          0
state                         0
course_title                  0
category_title                0
format                        0
completed_at                  0
transaction_amount            0
coupon_title              73578
coupon_discount_amount    73578
sale_price                 1112
pg                        15058
method                    13095
subcategory_title         19805
marketing_start_at        17955
dtype: int64

>결제 완료한 사람 중 환불하지 않은 사람만 판별 

In [16]:
# 결제가 완료된 사람의 데이터
df_pc = df[(df['type'] == 'PAYMENT') & (df['state'] == 'COMPLETED')]
df_pc.head()

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
2,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
4,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
8,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
10,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,NaN,CARD,NaN,2020-11-27
14,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02


In [17]:
# 환불이 완료된 사람의 데이터
df_rc =df[(df['type'] == 'REFUND') & (df['state'] == 'COMPLETED')]
df_rc.head()

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
651,444760.0,204122.0,REFUND,COMPLETED,퍼포먼스 마케터 되기 온라인 완주반 2기,마케팅,온라인 완주반,2021-03-29 09:24:43,-147000,NaN,NaN,404000.0,NaN,VBANK,NaN,NaT
652,472666.0,203178.0,REFUND,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-29 09:26:13,-96000,NaN,NaN,116000.0,NaN,CARD,영상 편집,2020-12-02
666,447177.0,203989.0,REFUND,COMPLETED,올인원 패키지 : 소금툰과 함께 캐릭터 드로잉/굿즈/임티/컷툰,디자인,올인원,2021-03-29 09:40:54,-79000,NaN,NaN,89000.0,NaN,CARD,일러스트,2021-02-19
672,373915.0,204264.0,REFUND,COMPLETED,데이터분석 프로젝트+자격증 온라인 완주반 3기,데이터사이언스,온라인 완주반,2021-03-29 09:44:01,-340000,NaN,NaN,340000.0,NaN,VBANK,NaN,2021-03-08
675,456086.0,NaN,REFUND,COMPLETED,프로모션 :연봉 앞자리가 바뀌는 프론트엔드 한끝세트 (3월 프리패스),프로그래밍,올인원,2021-03-29 09:47:17,-299000,NaN,NaN,299000.0,NaN,CARD,NaN,NaT


In [18]:
# 결제가 완료된 사람과 환불이 완료된 사람의 데이터를 합쳐준다.
df_total = pd.merge(df_pc,df_rc,how='outer')
df_total

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
1,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
3,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,NaN,CARD,NaN,2020-11-27
4,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56719,600073.0,206065.0,REFUND,COMPLETED,(프로모션 7/16-7/29) 초격차 패키지 : 한 번에 끝내는 Spring 완.전.판,프로그래밍,올인원,2021-09-24 14:34:50,-149000,NaN,NaN,179000.0,NAVER,CARD,백엔드 개발,2021-07-15
56720,645911.0,NaN,REFUND,COMPLETED,프로모션 : 콘텐츠 마케터 맥북 air 패스(9월 현물 프리패스),마케팅,올인원,2021-09-24 14:41:35,-1400000,NaN,NaN,1400000.0,INICIS,CARD,NaN,NaT
56721,506356.0,206720.0,REFUND,COMPLETED,용호수의 돈 버는 실전 영상 제작,영상/3D,올인원,2021-09-24 16:10:55,-259000,NaN,NaN,259000.0,NAVER,CARD,영상 편집,2021-08-17
56722,604077.0,205569.0,REFUND,COMPLETED,올인원 패키지 : 공여사와 무작정 풀어보는 엑셀 실무,업무 생산성,올인원,2021-09-24 17:09:30,-109000,NaN,NaN,129000.0,KAKAO,POINT,엑셀/VBA,2021-08-17


In [33]:
# 합친 데이터 중에서 중복된 customer_id를 제거해준다.
df_total= df_total.drop_duplicates(subset='customer_id')
df_total 

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
1,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
3,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,NaN,CARD,NaN,2020-11-27
4,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56702,131563.0,206179.0,REFUND,COMPLETED,The RED : Hadoop & Spark 아키텍처 딥다이브,데이터사이언스,RED,2021-09-23 19:05:38,-223200,NaN,NaN,279000.0,INICIS,CARD,데이터엔지니어링,2021-09-13
56708,642002.0,NaN,REFUND,COMPLETED,프로모션 : 디자인 맥북 air 패스(9월 현물 프리패스),디자인,올인원,2021-09-24 10:57:42,-1400000,NaN,NaN,1400000.0,INICIS,CARD,NaN,NaT
56709,666704.0,201849.0,REFUND,COMPLETED,올인원 패키지 : 그로스 마케팅,마케팅,올인원,2021-09-24 11:30:58,-86180,NaN,NaN,159000.0,INICIS,CARD,데이터마케팅,2019-11-25
56711,161993.0,205648.0,REFUND,COMPLETED,초격차 패키지 : 한번에 끝내는 금융ㆍ투자(IB) 실무,부동산/금융,올인원,2021-09-24 11:43:41,-299000,NaN,NaN,299000.0,INICIS,VBANK,금융/투자 실무,2021-08-02


In [37]:
# 남아있는 REFUND는 제공된 데이터 기간 전에 결제를 완료하고, 제공된 기간 내에 환불이 완료된 데이터이다.
# 정해진 기간내의 환불 비율을 알고싶으므로 PAYMENT만 골라낸다.
df_pay = df_total['type'] == 'PAYMENT'
df_pay = df_total[df_pay]
df_pay

,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,NaN,CARD,NaN,2021-03-04
1,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14
2,478033.0,203178.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:26:47,96000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,116000.0,NaN,CARD,영상 편집,2020-12-02
3,469860.0,203644.0,PAYMENT,COMPLETED,올인원 패키지 : 권오상의 금융 아카데미 - 권오상의 재무제표 분석 기초(평생소장),부동산/금융,올인원,2021-03-28 15:27:17,263000,NaN,NaN,263000.0,NaN,CARD,NaN,2020-11-27
4,478035.0,203720.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 프론트엔드 개발,프로그래밍,올인원,2021-03-28 15:31:01,91000,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,111000.0,NaN,CARD,프론트엔드 개발,2021-02-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52896,545804.0,NaN,PAYMENT,COMPLETED,프로모션 : 디자인 맥북 pro 패스(9월 현물 프리패스),디자인,올인원,2021-09-25 13:25:49,1800000,NaN,NaN,1800000.0,INICIS,CARD,NaN,NaT
52897,669238.0,204286.0,PAYMENT,COMPLETED,초격차 패키지 : 한번에 끝내는 3D 디자인 툴,디자인,올인원,2021-09-25 13:39:26,212000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,232000.0,INICIS,CARD,디자인툴,2021-04-23
52899,670891.0,203935.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 파이썬 웹 개발,프로그래밍,올인원,2021-09-25 14:01:26,159000,[미래투자금] 온라인 강의 2만원 할인쿠폰,20000.0,179000.0,INICIS,CARD,백엔드 개발,2021-05-31
52900,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15


>카테고리 컬럼 수정


In [38]:
# 1. 띄어쓰기 차이로 다른 강의로 구분된 '데이터사이언스'와 '데이터 사이언스'를 합쳐준다.
# 2. 디자인 카테고리에 서브 카테고리로 속해있는 '일러스트' 카테고리를 '디자인'카테고리에 편입시켜준다.
# 3. '교육','지식콘텐츠','비즈니스','파이낸스','관리' 카테고리는 현재 폐강된 카테고리들로 삭제해준다.
df['category_title'].value_counts().to_frame()

,category_title
프로그래밍,34430
데이터사이언스,14076
업무 생산성,13323
디자인,12009
영상/3D,10260
마케팅,9857
부동산/금융,5523
투자/재테크,2501
크리에이티브,2250
교육,1031


In [69]:
# 카테고리 변경
df_cate = df.replace({'category_title':{'데이터 사이언스':'데이터사이언스'}})
df_cate = df_cate.replace({'category_title':{'일러스트':'디자인'}})
df_cate['category_title'].value_counts().to_frame()

,category_title
프로그래밍,34430
데이터사이언스,14085
업무 생산성,13323
디자인,12010
영상/3D,10260
마케팅,9857
부동산/금융,5523
투자/재테크,2501
크리에이티브,2250


In [70]:
# 불필요한 카테고리 삭제
df_cate = df_cate[df_cate.category_title != '교육']
df_cate = df_cate[df_cate.category_title != '지식콘텐츠']
df_cate = df_cate[df_cate.category_title != '비즈니스']
df_cate = df_cate[df_cate.category_title != '파이낸스']
df_cate = df_cate[df_cate.category_title != '관리']

df_cate['category_title'].value_counts()

프로그래밍      34430
데이터사이언스    14085
업무 생산성     13323
디자인        12010
영상/3D      10260
마케팅         9857
부동산/금융      5523
투자/재테크      2501
크리에이티브      2250
Name: category_title, dtype: int64

In [ ]:
#일단 코드 저장
df[(df['coupon_title'].str.contains('웰컴')) | (df['coupon_title'].str.contains('WELCOME'))]